# ***Important***

**Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/172081/pages/assignment-instructions) page on Courseworks2 to learn the workflow for completing this project.**

<ins> Different from project 1 and 2</ins>, apart from the link to your notebook, you are also required to submit 2 additional files namely the collected data `data.pkl` and your chosen model checkpoint `dynamic.pth`.

## Project Setup

In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp23_project4.git

fatal: destination path 'mecs6616_sp23_project4' already exists and is not an empty directory.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp23_project4/* /content/

mv: cannot stat '/content/mecs6616_sp23_project4/*': No such file or directory


In [ ]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
from collections import defaultdict
class MPC:

  def __init__(self):
    self.control_horizon = 10 #H
    self.delta_u = 0.015
    self.prediction_horizon = 20 #N
    self.iterations = 40
    # Define other parameters here

  def compute_cost(self, dynamics, X, goal, U):

    cost=0

    Q = np.eye(dynamics.get_state_dim(),dynamics.get_state_dim())
    R = np.eye(dynamics.get_action_dim(),dynamics.get_action_dim())

    for i in range(len(X)):
      state = X[i]
      action = U[i]
      distance = np.linalg.norm(goal-dynamics.compute_fk(state))
      speed = np.linalg.norm(dynamics.compute_vel_ee(state))
      if(dynamics.num_links==3):
        cost += np.dot(np.dot(state, Q), np.transpose(state)) + 10*distance**2 + 5*speed**2
      else:
        cost += np.dot(np.dot(state, Q), np.transpose(state)) + 10*distance**2 + 10*speed**2

    return cost

  def compute_action(self, dynamics, state, goal, action):
    # Put your code here. You must return an array of shape (num_links, 1)

    U = np.ones((self.prediction_horizon, dynamics.get_action_dim()))
    for i in range(self.prediction_horizon):
      U[i, :] = action.reshape(-1)

    X = np.ones((self.prediction_horizon, dynamics.get_state_dim()))

    X[0,:]=state.reshape(-1)

    optimal_cost = self.compute_cost(dynamics, X, goal, U)

    optimal_action = action

    for iter in range(self.iterations):

      for i in range(self.prediction_horizon):
        U[i, :] = optimal_action.reshape(-1)

      for k in range(1, self.prediction_horizon):
        new_state = dynamics.advance(X[k-1, :].reshape((-1, 1)) , U[k-1, :].reshape((-1, 1)))
        X[k, :] = new_state.reshape(-1)
        if(np.linalg.norm(dynamics.compute_fk(X[k, :])-goal)<0.1):
          return optimal_action

      local_minimum_cost = float('inf')
      local_optimal_action = optimal_action

      ee = dynamics.compute_fk(X[k, :])

      for j in range(dynamics.get_action_dim()):
        temp_action = np.copy(action)
        if(((goal[1,0]-ee[1,0])/(goal[0,0]-ee[0,0]))>0):
          temp_action[j] += self.delta_u
          goal_forward=True
        else:
          temp_action[j] -= self.delta_u
          goal_forward=True

        for m in range(self.prediction_horizon):
          U[m, :] = temp_action.reshape(-1)

        for l in range(1, self.prediction_horizon):
          new_state = dynamics.advance(X[l-1, :].reshape((-1, 1)) , U[l-1, :].reshape((-1, 1)))
          X[l,:] = new_state.reshape(-1)
          if( np.linalg.norm(dynamics.compute_fk(X[l, :])- goal) <0.1):
            return optimal_action


        local_cost = self.compute_cost(dynamics, X, goal, U)

        if(local_cost<local_minimum_cost):
          local_minimum_cost = local_cost
          local_optimal_action = temp_action

        else:
          if(goal_forward):
            temp_action[j] -= self.delta_u
          else:
            temp_action[j] += self.delta_u


      action = np.copy(local_optimal_action)

      if(local_minimum_cost<optimal_cost):
        optimal_cost = local_minimum_cost
        optimal_action = local_optimal_action
      else:
        break

    # Don't forget to comment out the line below
    #raise NotImplementedError("MPC not implemented")
    best_action = optimal_action
    return best_action

# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.005
time_limit = 2.5
num_steps = round(time_limit/dt)

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

    action = controller.compute_action(arm.dynamics, state, goal, arm.action)
    arm.set_action(action)

At timestep 0: Distance to goal: 3.3970575502926055, Velocity of end effector: 5.539171676143414e-18
At timestep 10: Distance to goal: 3.37790842713045, Velocity of end effector: 0.14468818882016465
At timestep 20: Distance to goal: 3.3259891877281698, Velocity of end effector: 0.11222908849830618
At timestep 30: Distance to goal: 3.2369116849818833, Velocity of end effector: 0.0375700135741819
At timestep 40: Distance to goal: 3.096824033371842, Velocity of end effector: 0.08879318504302955
At timestep 50: Distance to goal: 2.896167573526578, Velocity of end effector: 0.24955081591324943
At timestep 60: Distance to goal: 2.6444993719787586, Velocity of end effector: 0.480271526899207
At timestep 70: Distance to goal: 2.3630764969156024, Velocity of end effector: 0.732493782206556
At timestep 80: Distance to goal: 2.0730204215511874, Velocity of end effector: 0.9795208992669018
At timestep 90: Distance to goal: 1.7917669343417033, Velocity of end effector: 1.1919289015638925
At timeste

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.33687098 -0.94155082], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.61202148 -0.79084114], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Partial Success:|
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.7675671  -1.76084481], Final velocity: [0.01365179]
score: 1.0/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.13978812 -1.29873619], Final velocity: [0.00198045]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.35772192 -2.3274941 ], Final velocity: [0.02471442]
score: 1.5/1.5
Test  2
Fail! :(
 Goal: [-2.10367746 -1.35075576], Final position: [-1.87079538 -1.36750008], Final velocity: [0.17413365]
score: 0/1.0
       
--------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step

	def forward(self, x):
		qddot = self.compute_qddot(x)
		state = x[:, :2*self.num_links]
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):
		# Your code goes here
		q = state[:, : self.num_links]
		qdot = state[:, self.num_links :]
		qdot_new = qdot + qddot*self.time_step
		q_new = q + qdot*self.time_step + 0.5*qddot*self.time_step**2
		new_state = np.concatenate((q_new, qdot_new), axis=1)
		return new_state

	def compute_qddot(self, x):
		pass


class Model2Link(Model):
    def __init__(self, time_step):
        super().__init__(2, time_step)
        # Your code goes here
        self.fc1 = nn.Linear(3*self.num_links, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, 8)
        self.fc7 = nn.Linear(8, self.num_links)

    def compute_qddot(self, x):
        # Your code goes here
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.relu(self.fc5(x))
        x = nn.functional.relu(self.fc6(x))
        x = self.fc7(x)
        return x.detach().numpy()


### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
# Teacher arm with 2 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()


def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.

  X = []
  Y = []
  for i in range(60):

    print("i", i)

    # Controller
    controller = MPC()
    gui = False

    if gui:
      renderer = Renderer()
      time.sleep(1)

    goal = np.zeros((2, 1))
    r = np.random.uniform(low=0.2, high=1.95)
    theta = np.random.uniform(low=np.pi, high=2.0*np.pi)
    goal[0,0] = r * np.cos(theta)
    goal[1,0] = r * np.sin(theta)
    arm.goal = goal

    dt = 0.01
    time_limit = 3
    num_steps = round(time_limit/dt)
    initial_state = np.zeros((arm.dynamics.get_state_dim(), 1))
    initial_state[0] = -math.pi / 2.0
    arm.reset()
    action = np.zeros((arm.dynamics.get_action_dim(), 1))
    arm.goal = goal
    arm.set_state(initial_state)
    for s in range(num_steps):

      t = time.time()

      state = arm.get_state()

      if s % controller.control_horizon == 0:
        action = controller.compute_action(dynamics_teacher, state, goal, action)
        arm.set_action(action)

      X.append(np.concatenate((state, action), axis=0))
      arm.advance()
      new_state = arm.get_state()
      Y.append(new_state)

      if gui:
        renderer.plot([(arm, "tab:blue")])
      time.sleep(max(0, dt - (time.time() - t)))



      #if( np.linalg.norm(arm.dynamics.compute_fk(state)- goal) <0.1):
      #      break


  X = np.hstack(X)
  Y = np.hstack(Y)
  print("shape of X is ", X.shape)
  print("shape of Y is ", Y.shape)
  return X, Y

In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))

i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
i 12
i 13
i 14
i 15
i 16
i 17
i 18
i 19
i 20
i 21
i 22
i 23
i 24
i 25
i 26
i 27
i 28
i 29
i 30
i 31
i 32
i 33
i 34
i 35
i 36
i 37
i 38
i 39
i 40
i 41
i 42
i 43
i 44
i 45
i 46
i 47
i 48
i 49
i 50
i 51
i 52
i 53
i 54
i 55
i 56
i 57
i 58
i 59
shape of X is  (6, 18000)
shape of Y is  (4, 18000)


### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].T.astype(np.float32)
    self.Y = data['Y'].T.astype(np.float32)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(model, train_loader):
  model.train()
	# ---
	# Your code goes here
	# ---

  learning_rate = 0.001
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  criterion = nn.MSELoss()
  total_items = 0
  training_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    features, labels = data
    optimizer.zero_grad()
    predicted_qddot = model.compute_qddot(features)
    state = features[:, :4]
    predicted_state = model.compute_next_state(state, predicted_qddot)
    loss = criterion(torch.Tensor(predicted_state), torch.Tensor(labels))
    loss.requires_grad = True
    loss.backward()
    optimizer.step()
    training_loss +=loss.item()
    total_items += labels.size(0)
  training_loss = training_loss/total_items
  print("training loss ", training_loss)
  return training_loss
  #pass

def test(model, test_loader):
    model.eval()
    # --
    # Your code goes here
    # --
    test_loss = 0
    criterion = nn.MSELoss()
    total_item = 0
    #dt = 0.01

    for i, data in enumerate(test_loader, 0):
        features, labels = data
        predicted_qddot = model.compute_qddot(features)
        state = features[:, :4]
        predicted_state=model.compute_next_state(state, predicted_qddot)
        loss = criterion(torch.Tensor(predicted_state), torch.Tensor(labels))
        test_loss += loss.item()
        total_item += labels.size(0)

    test_loss = test_loss/total_item
    print("test loss ", test_loss)
    # ---
    return test_loss


def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path
  model_path = None
  max_test_loss = 1e4
  model = Model2Link(0.01)
  best_epoch=-1

  datafile = 'dataset/data.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=3000, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=3000, shuffle=True)

  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  epochs=50
  for epoch in range(1, 1 + epochs):
    # --
    # Your code goes here
    # --
    print("in epoch ", epoch)
    train_one_epoch(model, train_loader)
    test_loss = test(model, test_loader)

    if(test_loss<max_test_loss):
      max_test_loss = test_loss
      best_epoch = epoch

    model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
    if not os.path.exists(os.path.join(model_dir, model_folder_name)):
      os.makedirs(os.path.join(model_dir, model_folder_name))
      torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))

    print("Best epoch:", best_epoch+1, "best loss", max_test_loss)
    #pass

  return model_path

In [ ]:
model_path = train_forward_model()

in epoch  1
training loss  1.7296144455435247e-08
test loss  2.9619522037036304e-08
Best epoch: 2 best loss 2.9619522037036304e-08
in epoch  2
training loss  1.7263194978214514e-08
test loss  2.7825904504020907e-08
Best epoch: 3 best loss 2.7825904504020907e-08
in epoch  3
training loss  1.7235465747944546e-08
test loss  2.9673990664175814e-08
Best epoch: 3 best loss 2.7825904504020907e-08
in epoch  4
training loss  1.730574316146279e-08
test loss  2.822178026286161e-08
Best epoch: 3 best loss 2.7825904504020907e-08
in epoch  5
training loss  1.7351895975379094e-08
test loss  2.668752030230179e-08
Best epoch: 6 best loss 2.668752030230179e-08
in epoch  6
training loss  1.72605622436903e-08
test loss  2.6317787463388716e-08
Best epoch: 7 best loss 2.6317787463388716e-08
in epoch  7
training loss  1.7266421410669054e-08
test loss  3.1981149327798956e-08
Best epoch: 7 best loss 2.6317787463388716e-08
in epoch  8
training loss  1.7273893667556372e-08
test loss  2.6462825594030113e-08
Best 

### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = Model2Link(0.01)
        checkpoint = torch.load(model_path)
        self.model.load_state_dict(checkpoint)
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            new_state = None

            self.model.eval()

            X = np.concatenate((state, action)).reshape(1, -1)

            predicted_qddot = self.model.compute_qddot(torch.FloatTensor(X))

            new_state=self.model.compute_next_state(state.reshape(1, -1), predicted_qddot.reshape(1,-1 ))

            return new_state
            # ---
        else:
            return state

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)


# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
model_path = "/content/models/2023-04-19_03-49-50/epoch_0013_loss_0.00000003/dynamics.pth"
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 2.7
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state.reshape(1, -1).reshape(-1), goal, action.T.reshape(1, -1).reshape(-1))
    arm.set_action(action)


## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
#model_path = 'dynamics.pth'
model_path = "/content/models/2023-04-19_03-49-50/epoch_0013_loss_0.00000003/dynamics.pth"
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Part2: EVALUATING CONTROLLER + LEARNED DYNAMICS
-----------------------------------------------
NUM_LINKS: 2
Test  1
Fail :(
 Goal: [-0.52056929 -0.58156835], Final position: [-0.03483414 -1.35635712], Final velocity: [0.05523066]
score: 0/0.5
Test  2
Fail :(
 Goal: [-0.31310822 -0.0363072 ], Final position: [0.43789598 0.20369306], Final velocity: [0.31031745]
score: 0/0.5
Test  3
Fail :(
 Goal: [-0.32408033 -0.03263632], Final position: [1.08674781 0.17847389], Final velocity: [0.43419629]
score: 0/0.5
Test  4
Partial success :|
 Goal: [-0.27871332 -0.94949408], Final position: [-0.48564895 -1.10423942], Final velocity: [0.4108524]
score: 0.3/0.5
Test  5
Fail :(
 Goal: [ 0.24024431 -0.46146338], Final position: [-0.79611122  0.28742758], Final velocity: [1.66734388]
score: 0/0.5
Test  6
Fail :(
 Goal: [-0.19292218 -0.10462694], Final position: [ 1.33000754 -0.23751822], Final velocity: [5.13513513]
score: 0/0.5
Test  7
Fail :(
 Goal: [ 0.9242372  -0.57481601], Final position: [ 0.280

Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy